# 멜론에서 발라드, 댄스 장르별로 2500개씩 크롤링하기

In [18]:
import requests
import lxml.html
import re
from bs4 import BeautifulSoup
import tqdm
import time

In [19]:
import itertools

In [20]:
from pandas import DataFrame, Series
import pandas as pd

In [21]:
url_base = 'http://www.melon.com/genre/song_listPaging.htm?startIndex={startindex}&pageSize=50&gnrCode={gnrcode}&dtlGnrCode=&orderBy=POP'

In [22]:
genre = ['GN0100', 'GN0200']  # 발라드=GN0100, 댄스='GN0200'

In [23]:
index_page = list(itertools.product(genre, range(1, 2500, 50)))

In [24]:
index_page[0]

('GN0100', 1)

In [25]:
detail_url = 'http://www.melon.com/song/detail.htm?songId={songid}'
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
song_url = []

for gnrcode, startindex in tqdm.tqdm_notebook(index_page):
    url = url_base.format(startindex=startindex, gnrcode=gnrcode)
    html = requests.get(url, headers=header).text
    soup = BeautifulSoup(html, 'html.parser')
    song_list = soup.select('a[href*=playSong]')
    
    for top_100 in song_list:
        play = top_100['href']
        detail = re.search(r"(\d+)'?\)", play)
        song_id = detail.group(1)
        song_url.append(detail_url.format(songid=song_id))

In [26]:
len(song_url)

5000

## 가사

In [27]:
genre = []
lyrics = []

for url in tqdm.tqdm_notebook(song_url):
    lyric_res = requests.get(url, headers=header)
    lyric_html = lyric_res.text
    soup_lr = BeautifulSoup(lyric_html, 'html.parser')
    
    gnr = soup_lr.select('.song_info.clfix dd')[3]
    genre.append(gnr.text)
    
    try:
        body = soup_lr.select('#d_video_summary')[0]
        body2 = re.sub(r'<.+->|<\/?\w+>', ' ', str(body))   #body.text를 하면 줄이 바뀔때마다 띄어쓰기가 안됨, 방법을 몰라서 그냥 <> 로 되있는걸 짜름
        lyrics.append(body2)
    except IndexError:
        lyrics.append('')
    
    df2 = DataFrame({'genre': genre,
                     'lyric': lyrics})
    
    time.sleep(5)

In [40]:
df2

,genre,lyric
0,Ballad,\r\n\t\t\t\t환상을 본 적이 있나요 전 있어요 아지랑이 같은 색깔이죠 텅...
1,Ballad,\r\n\t\t\t\t이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요 음 사랑한...
2,Ballad,\r\n\t\t\t\t비내리는 날엔 우산이 돼주고 어둠이 오면 빛이 돼줄게 추운 ...
3,Ballad,\r\n\t\t\t\t안녕 오랜만이야 물음표 없이 참 너다운 목소리 정해진 규칙처...
4,Ballad,\r\n\t\t\t\t더는 할 말이 없어 다른 이유도 어떤 변명도 난 의미 없단 ...
5,Ballad,\r\n\t\t\t\t조만간 얼굴 보자 언제 밥 한번 먹자 좋아 좋아 난 오...
6,Ballad,\r\n\t\t\t\t그리울 때 눈 감으면 더 잘 보이는 그런 사람 잊으려 하고 ...
7,Ballad,\r\n\t\t\t\t이건 비밀이야 아무에게도 고백하지 않았던 이야기를 들려주면...
8,Ballad,\r\n\t\t\t\t혼자 하는 말 혼자 자는 밤 혼자 있는 것 혼자 사는 것 그...
9,Ballad,\r\n\t\t\t\t그림 같은 집이 뭐 별거겠어요 어느 곳이든 그대가 있다면 ...


In [41]:
len(df2)

5000

## 저장

In [2]:
import csv
from pandas import read_csv

In [42]:
df2.to_csv('lyric_genre.csv', encoding='utf8')